
### Imports

In [1]:
%load_ext cython
%load_ext cythonmagic
%matplotlib inline
%matplotlib notebook

import pandas as pd
import numpy as np
import re, reprlib, sys
import matplotlib.pyplot as plt
from itertools import *
import random as ran
from logging import *
from pprint import *
from time import *
import shlex, subprocess

# import warnings
# warnings.filterwarnings('ignore')
# warnings.filterwarnings('ignore', category=DeprecationWarning)

import pandas_confusion as pconf
from pandas_confusion import *
from scipy.cluster.vq import *

import nltk as n
import nltk, nltk.classify.util, nltk.metrics, nltk.tokenize, nltk.stem
from nltk.corpus import *
from nltk.stem import *
from nltk.classify import *
from nltk.collocations import *
from nltk.metrics import *
from nltk.probability import *
from nltk.classify.scikitlearn import *
from nltk.tag.sequential import *
from nltk.tag import *
from nltk.tag.util import *
# n.download()

from sklearn_pandas import *

import sklearn as sk
from sklearn import *
from sklearn.feature_extraction.text import *
from sklearn.svm import *
from sklearn.preprocessing import *
from sklearn.cross_validation import *
from sklearn.pipeline import *
from sklearn.multiclass import *
from sklearn.datasets import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.feature_selection import *
from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.tree import *
from sklearn.grid_search import *
from sklearn.base import *
from sklearn.datasets.twenty_newsgroups import *
from sklearn.decomposition import *
from sklearn.feature_extraction import *
from sklearn.metrics import *
from sklearn.utils import *

sk.utils.check_random_state(5125)
ran.seed(5125)

/usr/local/lib/python3.4/dist-packages/IPython/extensions/cythonmagic.py:21: UserWarning: The Cython magic has been moved to the Cython package
  warnings.warn("""The Cython magic has been moved to the Cython package""")
/usr/local/lib/python3.4/dist-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


### R script adding new features to dataset 

In [2]:
# http://www.mango-solutions.com/wp/2015/10/integrating-python-and-r-part-ii-executing-r-from-python-and-vice-versa/
command = 'Rscript'
path2script = 'insert_features.R'

# Build subprocess command
cmd = [command, path2script]

subprocess.check_output(cmd, universal_newlines=True)

''

In [16]:
data_n = pd.read_csv("../raw_data/data_n.csv", parse_dates=True, infer_datetime_format=True)
data_n.head()

,Unnamed: 0,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,...,Number_of_Dots,Number_of_Commas,Number_of_Semicolons,Number_of_Colons,Average_Word_Length,Lexical_Diversity,Number_of_FunctionalWords,Number_of_Pronouns,Number_of_PROPNAMEs,SentimentNumeric
0,1,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,...,1,0,0,0,4.400,0.700,2,0,0,2
1,2,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3,3.15,3.25,4.4,n,y,...,1,0,0,0,3.615,0.577,10,1,0,0
2,3,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3,3.15,3.25,4.4,n,y,...,1,1,0,0,3.500,0.509,14,1,0,1
3,4,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3,3.15,3.25,4.4,n,y,...,1,0,0,0,3.667,0.611,4,0,0,2
4,5,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3,3.15,3.25,4.4,n,y,...,1,0,0,0,2.333,0.857,1,0,0,2


### Replace 'y' and 'n'   AND easy split

In [17]:
# http://stackoverflow.com/a/17702781
d = {'n': 0, 'y': 1}
data_n = data_n.replace(d)
data_n.head(n=1)

,Unnamed: 0,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,...,Number_of_Dots,Number_of_Commas,Number_of_Semicolons,Number_of_Colons,Average_Word_Length,Lexical_Diversity,Number_of_FunctionalWords,Number_of_Pronouns,Number_of_PROPNAMEs,SentimentNumeric
0,1,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,0,1,...,1,0,0,0,4.4,0.7,2,0,0,2


In [18]:
#norm:http://blog.yhat.com/posts/predicting-customer-churn-with-sklearn.html
# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# to_drop = ['State','Area Code','Phone','Churn?']
# churn_feat_space = data_n.drop(to_drop,axis=1)

In [33]:
neu = data_n[["#AUTHID","STATUS","cNEU", "sNEU", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]]
ext = data_n[["#AUTHID","STATUS","cEXT", "sEXT", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]]
agr = data_n[["#AUTHID","STATUS","cAGR", "sAGR","StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]]
con = data_n[["#AUTHID","STATUS","cCON", "sCON", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]]
opn = data_n[["#AUTHID","STATUS","cOPN", "sOPN", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]]

### Split stra. k-fold + selection of columns

In [20]:
# neu = data_n[["#AUTHID","STATUS","cNEU"]]
# train_neu, test_neu, y_train, y_test = sk.cross_validation.train_test_split(neu, neu["cNEU"], train_size = 0.66, stratify= neu["cNEU"])
# train_neu, test_neu, y_train, y_test 

In [34]:
train_feat_neu, test_feat_neu, train_class_neu, test_class_neu = sk.cross_validation.train_test_split(neu, neu["cNEU"], train_size = 0.66, stratify = neu["cNEU"],random_state= 5152)
print(train_feat_neu.shape, test_feat_neu.shape,train_class_neu.shape ,test_class_neu.shape)

neu_X = train_feat_neu.append(test_feat_neu)
neu_Y = train_class_neu.append(test_class_neu)

train_feat_ext, test_feat_ext, train_class_ext, test_class_ext = sk.cross_validation.train_test_split(ext, ext["cEXT"], train_size = 0.66, stratify = ext["cEXT"],random_state= 5152)
train_feat_agr, test_feat_agr, train_class_agr, test_class_agr = sk.cross_validation.train_test_split(agr, agr["cAGR"], train_size = 0.66, stratify = agr["cAGR"],random_state= 5152)
train_feat_con, test_feat_con, train_class_con, test_class_con = sk.cross_validation.train_test_split(con, con["cCON"], train_size = 0.66, stratify = con["cCON"],random_state= 5152)
train_feat_opn, test_feat_opn, train_class_opn, test_class_opn = sk.cross_validation.train_test_split(opn, opn["cOPN"], train_size = 0.66, stratify = opn["cOPN"],random_state= 5152)

(6545, 16) (3372, 16) (6545,) (3372,)


In [22]:
# mku = sk.pipeline.make_pipeline(SGDClassifier(), LinearSVC(),
#                                # BernoulliNB(), #MultinomialNB()
#                                # KNeighborsClassifier(),
#               RandomForestClassifier(n_jobs=-1),AdaBoostClassifier()) 

### Types of classifiers 
with their configuration

In [31]:
nick_names = ["ttSVC", "ttLinearSVC", "ttMNB", "ttBNB", "ttRF", "ttAD", "ttKNN"]
classifiers = [
    SVC(cache_size=500, kernel = "rbf", decision_function_shape = "ovr"),
    LinearSVC(),
    MultinomialNB(),
    BernoulliNB(),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, n_jobs=-1),
    AdaBoostClassifier(),
    KNeighborsClassifier()]

### Train + Test NEU

In [47]:
# print(classifiers, nick_names)
for name, clf in zip(nick_names, classifiers):
    clf.fit(train_feat_neu[["sNEU", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]], train_class_neu)
    y_pred = clf.predict(test_feat_neu[["sNEU", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]])
    print("\nFOR:", name)
    cm = pconf.BinaryConfusionMatrix(test_class_neu, y_pred)
    print(sk.metrics.average_precision_score(test_class_neu, y_pred))
    print(sk.metrics.accuracy_score(test_class_neu, y_pred, normalize=True))
    print(sk.metrics.mean_absolute_error(test_class_neu, y_pred))
    print(test_class_neu.shape, y_pred.shape)
    print(sk.metrics.classification_report(test_class_neu, y_pred))
    cm.print_stats()


FOR: ttSVC
0.834784451663
0.839264531435
0.160735468565
(3372,) (3372,)
             precision    recall  f1-score   support

          0       0.82      0.95      0.88      2108
          1       0.88      0.66      0.75      1264

avg / total       0.85      0.84      0.83      3372

population: 1144
P: 549
N: 595
PositiveTest: 341
NegativeTest: 803
TP: 174
TN: 428
FP: 167
FN: 375
TPR: 0.31693989071
TNR: 0.719327731092
PPV: 0.510263929619
NPV: 0.53300124533
FPR: 0.280672268908
FDR: 0.489736070381
FNR: 0.68306010929
ACC: 0.526223776224
F1_score: 0.391011235955
MCC: 0.0396121824983
informedness: 0.0362676218028
markedness: 0.0432651749488
prevalence: 0.479895104895
LRP: 1.12921697588
LRN: 0.949581226699
DOR: 1.18917365269
FOR: 0.46699875467

FOR: ttLinearSVC
0.982454161517
0.980723606168
0.0192763938316
(3372,) (3372,)
             precision    recall  f1-score   support

          0       0.97      1.00      0.98      2108
          1       0.99      0.96      0.97      1264

avg / t

In [25]:
eclf = VotingClassifier(estimators=[
        ("svc", ttSVC),
        ("linear_svc", ttLinearSVC)], voting='hard')


for clf, label in zip([ttSVC, ttLinearSVC], ["SVC", "LSVC"]):
    scores = cross_validation.cross_val_score(clf, neu_X[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]], neu_Y, cv=skf, scoring='accuracy')
    print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std()*2, label))
    print(scores)

NameError: name 'ttSVC' is not defined

In [26]:
skf = StratifiedKFold(neu["cNEU"], n_folds=10, shuffle=True, random_state = 5152)
print(skf) 

sklearn.cross_validation.StratifiedKFold(labels=[1 1 1 ..., 1 1 1], n_folds=10, shuffle=True, random_state=5152)


### Train + Test EXT

In [35]:
# print(classifiers, nick_names)
for name, clf in zip(nick_names, classifiers):
    clf.fit(train_feat_ext[["sEXT", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]], train_class_ext)
    y_pred = clf.predict(test_feat_ext[["sEXT", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]])
    print("\nFOR:", name)
    cm = pconf.ConfusionMatrix(test_class_ext, y_pred)
    cm.print_stats()


FOR: ttSVC
Confusion Matrix:

Predicted  0.0  1.0  __all__
Actual                      
0.0        460  242      702
1.0        266  175      441
__all__    726  417     1143


Overall Statistics:

Accuracy: 0.555555555556
95% CI: (0.5261992261437245, 0.58462526912446633)
No Information Rate: ToDo
P-Value [Acc > NIR]: 0.999999985895
Kappa: 0.0526284875184
Mcnemar's Test P-Value: ToDo


Class Statistics:

Classes                                        0          1
Population                                  1143       1143
P: Condition positive                        702        441
N: Condition negative                        441        702
Test outcome positive                        726        417
Test outcome negative                        417        726
TP: True Positive                            460        175
TN: True Negative                            175        460
FP: False Positive                           266        242
FN: False Negative                           242   